In [3]:
import pandas as pd
import sklearn as sk
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import RFE
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

from liver_functions import import_data
from liver_functions import one_hot_encode
from liver_functions import split_data
from joblib import dump, load
from sklearn.feature_selection import RFE
from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns
import pandas as pd
import numpy as np

def build_pipeline(classifier, isScaled):
    if isScaled:
        pipe = Pipeline(steps=[("imputer", SimpleImputer()),
                               ("scale", StandardScaler()),
                               ("clf", classifier)])#,
                               #("rfe", RFE(estimator=classifier))])
    else:
        pipe = Pipeline(steps=[("imputer", SimpleImputer()),
                               ("clf", classifier)])#,
                               #("rfe", RFE(estimator=classifier))])
        
    return pipe
    
def run_gridsearch(pipe, param_grid, num_folds, metric, X_train, y_train):
    search = GridSearchCV(pipe, param_grid, cv=num_folds, scoring=metric)
    search.fit(X_train, y_train)
    dump(search.best_estimator_, "test_model.pkl")
    
    return search.best_params_, search.best_score_, search.best_estimator_ 

"""
# Create the RFE object and rank each pixel
svc = SVC(kernel="linear", C=1)
rfe = RFE(estimator=svc, n_features_to_select=1, step=1)
rfe.fit(X, y)
ranking = rfe.ranking_.reshape(digits.images[0].shape)
"""

    
# Example
df = import_data("Indian Liver Patient Dataset (ILPD).csv")
df = one_hot_encode(df, "gender")
#display(df)
X_train, y_train, X_test, y_test = split_data(df, 0.2, "is_patient")
pipe = build_pipeline(KNeighborsClassifier(), True)
param_grid = dict(clf__n_neighbors=list(range(1,10)))
params, score, model = run_gridsearch(pipe, param_grid, 10, "f1", X_train, y_train)

In [4]:
sorted(sk.metrics.SCORERS.keys())

['accuracy',
 'adjusted_mutual_info_score',
 'adjusted_rand_score',
 'average_precision',
 'balanced_accuracy',
 'completeness_score',
 'explained_variance',
 'f1',
 'f1_macro',
 'f1_micro',
 'f1_samples',
 'f1_weighted',
 'fowlkes_mallows_score',
 'homogeneity_score',
 'jaccard',
 'jaccard_macro',
 'jaccard_micro',
 'jaccard_samples',
 'jaccard_weighted',
 'max_error',
 'mutual_info_score',
 'neg_brier_score',
 'neg_log_loss',
 'neg_mean_absolute_error',
 'neg_mean_gamma_deviance',
 'neg_mean_poisson_deviance',
 'neg_mean_squared_error',
 'neg_mean_squared_log_error',
 'neg_median_absolute_error',
 'neg_root_mean_squared_error',
 'normalized_mutual_info_score',
 'precision',
 'precision_macro',
 'precision_micro',
 'precision_samples',
 'precision_weighted',
 'r2',
 'recall',
 'recall_macro',
 'recall_micro',
 'recall_samples',
 'recall_weighted',
 'roc_auc',
 'roc_auc_ovo',
 'roc_auc_ovo_weighted',
 'roc_auc_ovr',
 'roc_auc_ovr_weighted',
 'v_measure_score']

In [ ]:
import unittest
import numpy as np
import pandas as pd
import string
import random
import os

from liver_functions import import_data
from liver_functions import one_hot_encode
from liver_functions import split_data
from liver_functions import make_correlation_heatmap

class TestFunctions(unittest.TestCase):
    
    def make_char_file(self):
        n = 5
        data = np.random.choice(list(string.ascii_lowercase),
                                     size=(n,n), replace = False)
        np.savetxt("test_data.csv", data, fmt= "%c", delimiter=",")
        return data
    
    def make_num_df(self):
        n = 5
        data = np.random.choice(100, size=(n,n), replace = False)
        return pd.DataFrame(data)
    
    def make_gender_df(self):
        n = 50
        gender = [random.choice(["Male", "Female"]) for i in range(n)]
        df = pd.DataFrame(gender, columns=["gender"])
        return df
    
    def test_heatmap(self):
        df = self.make_num_df()
        make_correlation_heatmap(df, "Test", "test.png")
        self.assertEqual(True, os.path.exists("test.png"))
         
    def test_one_hot(self):
        df = self.make_gender_df()
        df_enc = df.copy()
        df_enc["Male"] = df_enc["gender"].apply(lambda x: 1 if x == "Male" else 0)
        df_enc["Female"] = df_enc["gender"].apply(lambda x: 1 if x == "Female" else 0)
        df_enc.drop(columns = ["gender"],  inplace = True)
        pd.testing.assert_frame_equal(df_enc, one_hot_encode(df, "gender"),
                                      check_like = True, 
                                      check_dtype = False)

    def test_data_import(self):
        data = self.make_char_file()
        df = pd.DataFrame(data, columns= [x for x in data[0,:]])
        df = df.reindex(df.index.drop(0)).reset_index(drop=True)
        df_import = import_data("test_data.csv")
        pd.testing.assert_frame_equal(df, df_import, check_less_precise = 0) 

    
    def test_split(self):
        frac = 0.2
        data = self.make_char_file()
        n = len(data)
        df = pd.DataFrame(data, columns = [str(x) for x in range(n)])
        X_train, y_train, X_test, y_test = split_data(df, frac, str(n-1))
        
        self.assertEqual(np.shape(X_train), (round((1-frac)*n), n-1))
        self.assertEqual(np.shape(X_test), (round((frac)*n), n-1))
        self.assertEqual(np.shape(y_train), (round((1-frac)*n), ))
        self.assertEqual(np.shape(y_test), (round((frac)*n), ))
                                   
if __name__ == '__main__':        
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

In [ ]:
#df.loc[:, df["is_patient"] == 1]

var = "tot_proteins"
one = df.loc[df["is_patient"] == 1, [var]]
two = df.loc[df["is_patient"] == 2, [var]]

for var in df.columns:
    for label in df["is_patient"].unique():
        print(label)
        subset = df[df["is_patient"] == label]

        # Draw the density plot
        sns.distplot(subset[var], hist = False, kde = True,
                     kde_kws = {'shade': True,'linewidth': 3},
                     label = label)
    plt.show()

In [ ]:
#plt.scatter(df.index, df["tot_bilirubin"], c = df["is_patient"])
plt.boxplot([one["age"], two["age"]])

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize  = (10,10))
sns.pairplot(df, kind="reg", hue="is_patient")
plt.show()